In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

2024-06-19 00:44:02.498921: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-19 00:44:02.646638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 00:44:02.646672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 00:44:02.657507: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-19 00:44:02.736496: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-19 00:44:02.749600: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
sample_rate = 16000

In [4]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-06-19 00:44:07.849394: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-19 00:44:07.849821: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-19 00:44:07.961272: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-19 00:44:07.976502: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [5]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

change acctivation to softmax, make more neurons in first Dense

In [7]:
model = keras.Sequential([
    keras.layers.Input(shape=(48000,1)),
    keras.layers.Conv1D(filters=6, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling1D(pool_size=8),
    keras.layers.Conv1D(filters=8, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling1D(pool_size=10),
    keras.layers.Reshape((599*8,)),
    keras.layers.Dense(12, activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(2),
])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 47998, 6)          24        
                                                                 
 max_pooling1d (MaxPooling1  (None, 5999, 6)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 5997, 8)           152       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 599, 8)            0         
 g1D)                                                            
                                                                 
 reshape (Reshape)           (None, 4792)              0         
                                                                 
 dense (Dense)               (None, 12)                5

In [8]:

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# EPOCHS = 1
EPOCHS = 10
history = model.fit(
    train_dataset,
    epochs=EPOCHS
)

Epoch 1/10
353/353 [==============================] - 59s 162ms/step - loss: 0.3329 - accuracy: 0.8396
Epoch 2/10
353/353 [==============================] - 42s 116ms/step - loss: 0.2300 - accuracy: 0.9071
Epoch 3/10
353/353 [==============================] - 42s 117ms/step - loss: 0.1914 - accuracy: 0.9233
Epoch 4/10
353/353 [==============================] - 44s 122ms/step - loss: 0.1602 - accuracy: 0.9416
Epoch 5/10
353/353 [==============================] - 45s 126ms/step - loss: 0.1423 - accuracy: 0.9447
Epoch 6/10
353/353 [==============================] - 45s 125ms/step - loss: 0.1184 - accuracy: 0.9574
Epoch 7/10
353/353 [==============================] - 43s 121ms/step - loss: 0.1036 - accuracy: 0.9623
Epoch 8/10
353/353 [==============================] - 44s 121ms/step - loss: 0.0959 - accuracy: 0.9647
Epoch 9/10
353/353 [==============================] - 43s 120ms/step - loss: 0.0845 - accuracy: 0.9685
Epoch 10/10
353/353 [==============================] - 49s 136ms/step - l

In [9]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(model, x_test_np, y_test_np, model_format="keras")


Validation dataset:
44/44 [==============================] - 1s 22ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 95.07%
Recall: 88.35%
Precision: 96.98%
F1-score: 92.46%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9229463464959073

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9275873181529032

Test dataset:
44/44 [==============================] - 1s 23ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9384192313336935

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9404208800514493


In [10]:
print("For Test dataset:")
evaluate_time_of_prediction(model, x_test_np, y_test_np, model_format="keras", show_prediction_evaluation=True)


For Test dataset:
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Time to make a prediction for a single data point
Mean: 0.046 seconds
Max: 0.178 seconds
Min: 0.041 seconds


In [6]:
BASE_MODEL_FILE_NAME = "time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_baseline.keras"
# model.save(BASE_MODEL_FILE_NAME)
# print("Model file name: ", BASE_MODEL_FILE_NAME)
# convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

In [7]:
model_loaded = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)

In [8]:
# print("Validation dataset:")
# (
#     y_pred_val, 
#     non_overlap_patritions_f1_scores_val, 
#     bootstrap_patritions_f1_scores_val,
# ) = predict_and_print_full_results(model_loaded, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(model_loaded, x_test_np, y_test_np, model_format="keras")



Test dataset:
 1/44 [..............................] - ETA: 6s

2024-06-19 00:44:44.699999: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 267456000 exceeds 10% of free system memory.
2024-06-19 00:44:44.916655: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36862464 exceeds 10% of free system memory.
2024-06-19 00:44:44.974273: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36862464 exceeds 10% of free system memory.


 8/44 [====>.........................] - ETA: 0s

2024-06-19 00:44:44.999320: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36862464 exceeds 10% of free system memory.
2024-06-19 00:44:45.027759: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36862464 exceeds 10% of free system memory.


44/44 [==============================] - 1s 23ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9384192313336935

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.938042035485987


In [16]:
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "KB")

File size: 712.659 Kilobytes
